# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 77 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.13789


extracting tarball to tmp_2208.13789...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.13791


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.13789/CO_OH_AA.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.14333


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.14293/BP_maps.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'BP_acknowledgments.tex' from 'tmp_2208.14293/BP_acknowledgments.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'BP13_authors.tex' from 'tmp_2208.14293/BP13_authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/p

extracting tarball to tmp_2208.14333...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.14333/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13791-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13791) | **The Pristine Inner Galaxy Survey (PIGS) V: a chemo-dynamical  investigation of the early assembly of the Milky Way with the most metal-poor  stars in the bulge**  |
|| Federico Sestito, et al. -- incl., <mark>Jonay I. González Hernández</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *22 pages + appendix, 14 Figures + 3 in the appendix, submitted to MNRAS*|
|**Abstract**| The investigation of the metal-poor tail in the Galactic bulge provides unique information on the early Milky Way assembly and evolution. A chemo-dynamical analysis of 17 very metal-poor stars (VMP, [Fe/H] $<-2.0$) selected from the Pristine Inner Galaxy Survey was carried out based on Gemini/GRACES spectra. The chemical abundances of $\alpha-$elements (Mg, Ca, and Ti), odd-Z (Na, K, and Sc), Fe-peak (Cr and Ni), and neutron-capture (Ba) elements are determined from model atmosphere analyses. The chemistry suggests that the majority of our stars are very similar to metal-poor stars in the Galactic halo. Orbits calculated from {\it Gaia} EDR3 imply these stars are brought into the bulge during the earliest Galactic assembly. Most of our stars have large [Na,Ca/Mg] abundances, and thus show little evidence of enrichment by pair-instability supernovae. Two of our stars (P171457, P184700) have chemical abundances compatible with second-generation globular cluster stars, suggestive of the presence of ancient and now dissolved globular clusters in the inner Galaxy. One of them (P171457) is extremely metal-poor ([Fe/H] $<-3.0$) and well below the metallicity floor of globular clusters, which supports the growing evidence for the existence of lower-metallicity globular clusters in the early Universe. A third star (P180956, [Fe/H] $\sim-2$) has low [Na,Ca/Mg] and very low [Ba/Fe] for its metallicity, which are consistent with formation in a system polluted by only one or a few low-mass supernovae. Interestingly, its orbit is confined to the Galactic plane, like other very metal-poor stars found in the literature, which have been associated with the earliest building blocks of the Milky Way. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14181-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14181) | **A multifrequency characterization of the extragalactic hard X-ray sky**  |
|| Matej Kosiba, et al. -- incl., <mark>Harold Andres Peña-Herazo</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *19 pages, 7 tables, 17 figures, accepted for publication in A&A - pre-proofs version, catalog tables will be available online*|
|**Abstract**| Nowadays we know that the origin of the Cosmic X-ray Background (CXB) is due to the integrated emission of nearby active galactic nuclei. Thus, to obtain a precise estimate of the contribution of different source classes to the CXB it is crucial to fully characterize the hard X-ray sky. We present a multifrequency analysis of all sources listed in the 3d release of the Palermo Swift-BAT hard X-ray catalog (3PBC) with the goal of (i) identifying and classifying the largest number of sources adopting multifrequency criteria, with particular emphasis on extragalactic populations and (ii) extracting Seyfert galaxies to present here the 2nd release of the Turin-SyCAT catalog. We outline a classification scheme based on radio, infrared and optical criteria that allows us to distinguish between unidentified and unclassified hard X-ray sources, and classify and classify the remaining ones discriminating between Galactic and extragalactic classes. Our revised version of the 3PBC lists 1176 classified, 820 extragalactic, and 356 Galactic, 199 unclassified, and 218 unidentified sources. According to our analysis, the hard X-ray sky is mainly populated by Seyfert galaxies and blazars. For the Seyfert galaxies, we present the 2nd release of the Turin-SyCAT including a total of 633 Seyfert galaxies, with 282 new sources, which is an 80% increment from the previous release. We confirm, that there is no clear difference between the flux distribution of the infrared-to-hard X-ray flux ratio of Seyfert galaxies Type 1 and Type 2. However, we confirm a significant trend between the mid-IR flux and hard X-ray flux. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13789-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13789) | **The formation of CO$_2$ through consumption of gas-phase CO on vacuum-UV  irradiated water ice**  |
|| J. Terwisscha van Scheltinga, et al. -- incl., <mark>A.D. Bosman</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| [Abridged] Observations of protoplanetary disks suggest that they are depleted in gas-phase CO. It has been posed that gas-phase CO is chemically consumed and converted into less volatile species through gas-grain processes. Observations of interstellar ices reveal a CO$_2$ component within H$_2$O ice suggesting co-formation. The aim of this work is to experimentally verify the interaction of gas-phase CO with solid-state OH radicals above the sublimation temperature of CO. Amorphous solid water (ASW) is deposited at 15 K and followed by vacuum-UV (VUV) irradiation to dissociate H$_2$O and create OH radicals. Gas-phase CO is simultaneously admitted and only adsorbs with a short residence time on the ASW. Products in the solid state are studied with infrared spectroscopy and once released into the gas phase with mass spectrometry. Results show that gas-phase CO is converted into CO$_2$, with an efficiency of 7-27%, when interacting with VUV irradiated ASW. Between 40 and 90 K, CO$_2$ production is constant, above 90 K, O$_2$ production takes over. In the temperature range of 40-60 K, the CO$_2$ remains in the solid state, while at temperatures $\geq$ 70 K the formed CO$_2$ is released into the gas phase. We conclude that gas-phase CO reacts with solid-state OH radicals above its sublimation temperature. This gas-phase CO and solid-state OH radical interaction could explain the observed CO$_2$ embedded in water-rich ices. It may also contribute to the observed lack of gas-phase CO in planet-forming disks, as previously suggested. Our experiments indicate a lower water ice dissociation efficiency than originally adopted in model descriptions of planet-forming disks and molecular clouds. Incorporation of the reduced water ice dissociation and increased binding energy of CO on a water ice surfaces in these models would allow investigation of this gas-grain interaction to its full extend. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14276-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14276) | **BeyondPlanck XI. Bayesian CMB analysis with sample-based end-to-end  error propagation**  |
|| L. P. L. Colombo, et al. -- incl., <mark>B. Hensley</mark>, <mark>D. Herman</mark>, <mark>G. Maggio</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *26 pages, 24 figures. Submitted to A&A. Part of the BeyondPlanck paper suite*|
|**Abstract**| We present posterior sample-based cosmic microwave background (CMB) constraints from Planck LFI and WMAP observations derived through global end-to-end Bayesian processing. We use these samples to study correlations between CMB, foreground, and instrumental parameters, and we identify a particularly strong degeneracy between CMB temperature fluctuations and free-free emission on intermediate angular scales, which is mitigated through model reduction, masking, and resampling. We compare our posterior-based CMB results with previous Planck products, and find generally good agreement, but with higher noise due to exclusion of HFI data. We find a best-fit CMB dipole amplitude of $3362.7\pm1.4{\mu}K$, in excellent agreement with previous Planck results. The quoted uncertainty is derived directly from the sampled posterior distribution, and does not involve any ad hoc contribution for systematic effects. Similarly, we find a temperature quadrupole amplitude of $\sigma^{TT}_2=229\pm97{\mu}K^2$, in good agreement with previous results in terms of the amplitude, but the uncertainty is an order of magnitude larger than the diagonal Fisher uncertainty. Relatedly, we find lower evidence for a possible alignment between $\ell = 2$ and $\ell = 3$ than previously reported due to a much larger scatter in the individual quadrupole coefficients, caused both by marginalizing over a more complete set of systematic effects, and by our more conservative analysis mask. For higher multipoles, we find that the angular temperature power spectrum is generally in good agreement with both Planck and WMAP. This is the first time the sample-based asymptotically exact Blackwell-Rao estimator has been successfully established for multipoles up to $\ell\le600$, and it now accounts for the majority of the cosmologically important information. Cosmological parameter constraints are presented in a companion paper. (Abriged) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14293-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14293) | **BeyondPlanck X. Planck LFI frequency maps with sample-based error  propagation**  |
|| A. Basyrov, et al. -- incl., <mark>B. Hensley</mark>, <mark>D. Herman</mark>, <mark>G. Maggio</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *32 pages, 23 figures, data available from this https URL*|
|**Abstract**| We present Planck LFI frequency sky maps derived within the BeyondPlanck framework. This framework draws samples from a global posterior distribution that includes instrumental, astrophysical and cosmological parameters, and the main product is an entire ensemble of frequency sky map samples. This ensemble allows for computationally convenient end-to-end propagation of low-level instrumental uncertainties into higher-level science products. We show that the two dominant sources of LFI instrumental systematic uncertainties are correlated noise and gain fluctuations, and the products presented here support - for the first time - full Bayesian error propagation for these effects at full angular resolution. We compare our posterior mean maps with traditional frequency maps delivered by the Planck collaboration, and find generally good agreement. The most important quality improvement is due to significantly lower calibration uncertainties in the new processing, as we find a fractional absolute calibration uncertainty at 70 GHz of $\delta g_{0}/g_{0} =5 \cdot 10^{-5}$, which is nominally 40 times smaller than that reported by Planck 2018. However, the original Planck 2018 estimate has a non-trivial statistical interpretation, and this further illustrates the advantage of the new framework in terms of producing self-consistent and well-defined error estimates of all involved quantities without the need of ad hoc uncertainty contributions. We describe how low-resolution data products, including dense pixel-pixel covariance matrices, may be produced directly from the posterior samples without the need for computationally expensive analytic calculations or simulations. We conclude that posterior-based frequency map sampling provides unique capabilities in terms of low-level systematics modelling and error propagation, and may play an important role for future CMB B-mode experiments. (Abridged.) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14333-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14333) | **Detection of GeV emission from an ultra-long gamma-ray burst with the  Fermi Large Area Telescope**  |
|| Yi-Yun Huang, et al. -- incl., <mark>Hai-Ming Zhang</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *11 pages, 3 figures, 2 tables, submitted to ApJ*|
|**Abstract**| GRB 220627A, detected by Fermi GBM, shows two episodes of gamma-ray emission, which are separated by a {$\sim$700} s-long quiescent phase. Due to similar temporal shapes and spectra in the two episodes, GRB 220627A is speculated to be a gravitationally-lensed GRB. We analyze the Fermi-LAT data and find that about 49 gamma-ray photons above 100 MeV come from the GRB during the first episode, while there is no GeV photons detected in the second episode. Based on the broad-band spectral study of the two episodes, the gravitationally-lensing scenario can be ruled out at a high confidence level and we thus conclude that GRB 220627A is an intrinsically ultra-long GRB with the prompt burst emission lasting longer than 1000 s. It is then the first case that GeV emission is detected from an ultra-long GRB. We find that a short spike seen in the LAT light curve is also present in GBM detectors that see the burst, suggesting a common internal region of emission across the entire Fermi energy range. The detection of a 15.7-GeV photon during the early prompt phase places a lower limit of $\Gamma\ge300$ on the bulk Lorentz factor of the GRB ejecta. The constraint on the bulk Lorentz factor could shed light on the origin of ultra-long GRBs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14159-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14159) | **Development of the characterization methods without electrothermal  feedback for TES bolometers for CMB measurements**  |
|| Yume Nishinomiya, et al. -- incl., <mark>Adrian Lee</mark>, <mark>Heather McCarrick</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *7 pages, 6 figures, LTD19 Conference Proceedings*|
|**Abstract**| Superconducting Transition-Edge Sensor (TES) bolometers are used for cosmic microwave background (CMB) observations. We used a testbed to evaluate the thermal performance of TES bolometers in regard to the saturation power Psat and intrinsic thermal time constant tau0. We developed an evaluation method that is complementary to methods with electrothermal feedback. In our method, the antenna termination resistor of the bolometer is directly biased with DC or AC electric power to simulate optical power, and the TES is biased with small power, which allows Psat and tau0 to be determined without contribution from the negative electrothermal feedback. We describe the method and results of the measurement using it. We evaluated Psat of five samples by applying DC power and confirmed the overall trend between Psat and the inverse leg length. We evaluated tau0 of the samples by applying DC plus AC power, and the measured value was reasonable in consideration of the expected values of other TES parameters. This evaluation method enables us to verify whether a TES has been fabricated with the designed values and to provide feedback for fabrication for future CMB observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.13806-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.13806) | **Depletion of gaseous CO in protoplanetary disks by  surface-energy-regulated ice formation**  |
|| Diana Powell, Peter Gao, Ruth Murray-Clay, <mark>Xi Zhang</mark> |
|*Appeared on*| *2022-08-31*|
|*Comments*| *Published in Nature Astronomy (2022), includes supplementary material*|
|**Abstract**| Empirical constraints of fundamental properties of protoplanetary disks are essential for understanding planet formation and planetary properties (1,2). Carbon monoxide (CO) gas is often used to constrain disk properties (3). However, estimates show that the CO gas abundance in disks is depleted relative to expected values (4,5,6,7), and models of various disk processes impacting the CO abundance could not explain this depletion on observed 1Myr timescales (8,9,10,11,12,13,14). Here we demonstrate that surface energy effects on particles in disks, such as the Kelvin effect, that arise when ice heterogeneously nucleates onto an existing particle can efficiently trap CO in its ice phase. In previous ice formation models, CO gas was released when small ice-coated particles were lofted to warmed disk layers. Our model can reproduce the observed abundance, distribution and time evolution of gaseous CO in the four most studied protoplanetary disks (7). We constrain the solid and gaseous CO inventory at the midplane and disk diffusivities and resolve inconsistencies in estimates of the disk mass -- three crucial parameters that control planetary formation. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.13791.md
    + _build/html/tmp_2208.13791/././plots/kine.png
    + _build/html/tmp_2208.13791/././plots/bah.png
    + _build/html/tmp_2208.13791/././plots/teff_DR3.png
    + _build/html/tmp_2208.13791/././plots/logg_DR3.png
    + _build/html/tmp_2208.13791/././plots/met_DR3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\ubar}[1]{\underaccent{\bar}{#1}}$
$\newcommand{\fmmm}[1]{\mbox{#1}}$
$\newcommand{\scnd}{\mbox{\fmmm{"}\hskip-0.3em .}}$
$\newcommand{\scnp}{\mbox{\fmmm{"}}}$
$\newcommand{\mcnd}{\mbox{\fmmm{'}\hskip-0.3em .}}$
$\newcommand{\ang}{\textup{\AA} }$
$\newcommand{\kv}[1]{\textcolor{red}{#1}}$
$\newcommand{\pmu}{\muas yr^{-1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{log g}$
$\newcommand{\Pristine}{{\it Pristine}}$
$\newcommand{\Gaia}{{\it Gaia}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\lta}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\erf}{\mathop{\rm erf}\nolimits}$
$\newcommand{\sech}{ \mathop{\rm sech}\nolimits}$
$\newcommand{\csch}{ \mathop{\rm csch}\nolimits}$
$\newcommand{\arcsinh}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{\arccosh}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{\arctanh}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{\arccoth}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{\arcsech}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{\arccsch}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{\arccot}{\mathop{\rm arccot}\nolimits}$
$\newcommand{\arcsec}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{\arccsc}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{\ylm}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{\ylmp}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{\real}{\Re e}$
$\newcommand{\imag}{\Im m}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\kms}{{\rm km   s^{-1}}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\lsun}{{\rm L_\odot}}$
$\newcommand{\rsun}{{\rm R_\odot}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\g}{{\rm g}}$
$\newcommand{\om}{\Omega_0}$
$\newcommand{\}{ca}$
$\newcommand{\}{magnitude}$
$\newcommand{\kr}{{\cal K}_r}$
$\newcommand{\kz}{{\cal K}_z}$
$\newcommand{\kzz}{{\cal K}_z(z)}$
$\newcommand{\mss}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{\msss}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\CompactFigs}{0}$
$\newcommand{\UseFigs}{1}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degg}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{\sec}{\hbox{"\hskip-3pt .}}$
$\newcommand{\half}{{\scriptstyle{1\over2}}}$
$\newcommand{\s}{\ifmmode \widetilde \else \~\fi}$
$\newcommand{\=}{\overline}$
$\newcommand{\scre}{{\cal E}}$
$\newcommand{\}{spose}$
$\newcommand{\larrow}{\leftarrow}$
$\newcommand{\rarrow}{\rightarrow}$
$\newcommand{\llangle}{\langle\langle}$
$\newcommand{\rrangle}{\rangle\rangle}$
$\newcommand{\etal}{{\it et al. }}$
$\newcommand{\cf}{{\it cf. }}$
$\newcommand{\eg}{{e.g., }}$
$\newcommand{\ie}{{i.e., }}$
$\newcommand{\lta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13C}}}$
$\newcommand{\gta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13E}}}$
$\newcommand{\Dt}{\spose{\raise 1.5ex\hbox{\hskip3pt\mathchar"201}}}$
$\newcommand{\dt}{\spose{\raise 1.0ex\hbox{\hskip2pt\mathchar"201}}}$
$\newcommand{\del}{\nabla}$
$\newcommand{\delv}{\bb\nabla}$
$\newcommand{\jla}{J_{\lambda}}$
$\newcommand{\jmu}{J_{\mu}}$
$\newcommand{\jnu}{J_{\nu}}$
$\newcommand{\pomega}{\varpi}$
$\newcommand{\sigla}{\sigma_{\lambda}}$
$\newcommand{\sigmu}{\sigma_{\mu}}$
$\newcommand{\signu}{\sigma_{\nu}}$
$\newcommand{\dotsfill}{\leaders\hbox to 1em{\hss.\hss}\hfill}$
$\newcommand{\sun}{\odot}$
$\newcommand{\earth}{\oplus}$
$\newcommand{\Gyr}{{\rm Gyr}}$
$\newcommand{\FeH}{{\rm[Fe/H]}}$
$\newcommand{\kmsd}{{\rm km/s/degree}}$
$\newcommand{\ione}{{~\sc i}}$
$\newcommand{\ii}{{~\sc ii}}$
$\newcommand{\}{Fed}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\ubar$}[1]{\underaccent{\bar}{#1}}$
$\newcommand{$\fmmm$}[1]{\mbox{#1}}$
$\newcommand{$\scnd$}{\mbox{$\fmmm${"}\hskip-0.3em .}}$
$\newcommand{$\scnp$}{\mbox{$\fmmm${"}}}$
$\newcommand{$\mcnd$}{\mbox{$\fmmm${'}\hskip-0.3em .}}$
$\newcommand{$\ang$}{\textup{\AA} }$
$\newcommand{$\kv$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\pmu$}{\muas yr^{-1}}$
$\newcommand{$\teff$}{T_{\rm eff}}$
$\newcommand{$\logg$}{log g}$
$\newcommand{$\Pristine$}{{\it Pristine}}$
$\newcommand{$\Gaia$}{{\it Gaia}}$
$\newcommand{$\ltsima$}{\; \buildrel < \over $\s$im \;}$
$\newcommand{$\lta$}{\lower.5ex\hbox{$\ltsima$}}$
$\newcommand{$\gtsima$}{\; \buildrel > \over $\s$im \;}$
$\newcommand{$\simgt$}{\lower.5ex\hbox{$\gtsima$}}$
$\newcommand{$\erf$}{\mathop{\rm erf}\nolimits}$
$\newcommand{$\sech$}{ \mathop{\rm sech}\nolimits}$
$\newcommand{$\csch$}{ \mathop{\rm csch}\nolimits}$
$\newcommand{$\arcsinh$}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{$\arccosh$}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{$\arctanh$}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{$\arccoth$}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{$\arcsec$h}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{$\arccsch$}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{$\arccot$}{\mathop{\rm arccot}\nolimits}$
$\newcommand{$\arcsec$}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{$\arccsc$}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{$\ylm$}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{$\ylm$p}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{$\real$}{\Re e}$
$\newcommand{$\imag$}{\Im m}$
$\newcommand{$\km$}{{\rm km}}$
$\newcommand{$\km$s}{{\rm km   s^{-1}}}$
$\newcommand{$\mas$}{{\rm mas}}$
$\newcommand{$\mas$yr}{{\rm mas/yr}}$
$\newcommand{$\kpc$}{{\rm kpc}}$
$\newcommand{$\mpc$}{{\rm Mpc}}$
$\newcommand{$\msun$}{{\rm M_\odot}}$
$\newcommand{$\lsun$}{{\rm L_\odot}}$
$\newcommand{$\rsun$}{{\rm R_\odot}}$
$\newcommand{$\pc$}{{\rm pc}}$
$\newcommand{$\cm$}{{\rm cm}}$
$\newcommand{$\yr$}{{\rm yr}}$
$\newcommand{$\au$}{{\rm AU}}$
$\newcommand{$\g$}{{\rm g}}$
$\newcommand{$\om$}{\Omega_0}$
$\newcommand{\}{ca}$
$\newcommand{\}{magnitude}$
$\newcommand{$\kr$}{{\cal K}_r}$
$\newcommand{$\kz$}{{\cal K}_z}$
$\newcommand{$\kz$z}{{\cal K}_z(z)}$
$\newcommand{$\mss$}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{$\mss$s}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{$\yr$}{{\rm yr}}$
$\newcommand{$\CompactFigs$}{0}$
$\newcommand{$\UseFigs$}{1}$
$\newcommand{$\deg$}{^\circ}$
$\newcommand{$\deg$g}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{$\sec$}{\hbox{"\hskip-3pt .}}$
$\newcommand{$\half$}{{$\s$criptstyle{1\over2}}}$
$\newcommand{$\s$}{\ifmmode \widetilde \else \~\fi}$
$\newcommand{\=}{\overline}$
$\newcommand{$\s$cre}{{\cal E}}$
$\newcommand{\}{spose}$
$\newcommand{$\larrow$}{\leftarrow}$
$\newcommand{$\rarrow$}{\rightarrow}$
$\newcommand{$\llangle$}{\langle\langle}$
$\newcommand{$\rrangle$}{\rangle\rangle}$
$\newcommand{$\etal$}{{\it et al. }}$
$\newcommand{$\cf$}{{\it cf. }}$
$\newcommand{$\eg$}{{e.g., }}$
$\newcommand{$\ie$}{{i.e., }}$
$\newcommand{$\lta$}{\mathrel{$\s$pose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13C}}}$
$\newcommand{$\g$ta}{\mathrel{$\s$pose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13E}}}$
$\newcommand{$\Dt$}{$\s$pose{\raise 1.5ex\hbox{\hskip3pt\mathchar"201}}}$
$\newcommand{$\dt$}{$\s$pose{\raise 1.0ex\hbox{\hskip2pt\mathchar"201}}}$
$\newcommand{$\del$}{\nabla}$
$\newcommand{$\del$v}{\bb\nabla}$
$\newcommand{$\jla$}{J_{\lambda}}$
$\newcommand{$\jmu$}{J_{\mu}}$
$\newcommand{$\jnu$}{J_{\nu}}$
$\newcommand{$\pomega$}{\varpi}$
$\newcommand{$\s$igla}{$\s$igma_{\lambda}}$
$\newcommand{$\s$igmu}{$\s$igma_{\mu}}$
$\newcommand{$\s$ignu}{$\s$igma_{\nu}}$
$\newcommand{$\dotsfill$}{\leaders\hbox to 1em{\hss.\hss}\hfill}$
$\newcommand{$\s$un}{\odot}$
$\newcommand{$\earth$}{\oplus}$
$\newcommand{$\Gyr$}{{\rm Gyr}}$
$\newcommand{$\FeH$}{{\rm[Fe/H]}}$
$\newcommand{$\km$sd}{{\rm km/s/degree}}$
$\newcommand{$\ion$e}{{~$\s$c i}}$
$\newcommand{$\ii$}{{~$\s$c ii}}$
$\newcommand{\}{Fed}$</div>



<div id="title">

# The Pristine Inner Galaxy Survey (PIGS) V: a chemo-dynamical investigation of the early assembly of the Milky Way with the most metal-poor stars in the bulge

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.13791-b31b1b.svg)](https://arxiv.org/abs/2208.13791)<mark>Appeared on: 2022-08-31</mark> - _22 pages + appendix, 14 Figures + 3 in the appendix, submitted to MNRAS_

</div>
<div id="authors">

Federico Sestito, et al. -- incl., <mark><mark>Jonay I. González Hernández</mark></mark>

</div>
<div id="abstract">

**Abstract:** The investigation of the  metal-poor tail in the Galactic bulge provides unique information on the early Milky Way assembly and evolution. A chemo-dynamical analysis of 17 very metal-poor stars (VMP,$\FeH$$<-2.0$) selected from the Pristine Inner Galaxy Survey was carried out  based on  Gemini/GRACES spectra. The chemical abundances of$\alpha-$elements (Mg, Ca, and Ti), odd-Z  (Na, K, and Sc), Fe-peak (Cr and Ni), and neutron-capture  (Ba) elements are determined from model atmosphere analyses. The chemistry suggests that the majority of our stars are very similar to metal-poor stars in the Galactic halo.  Orbits calculated from{\it Gaia}EDR3 imply these stars are brought into the bulge during the earliest Galactic assembly. Most of our stars have large[Na,Ca/Mg]abundances, and thus show little evidence of enrichment by pair-instability supernovae.Two of our stars (P171457, P184700) have chemical abundances compatible with second-generation globular cluster stars, suggestive of the presence of ancient and now dissolved globular clusters in the inner Galaxy. One of them (P171457) is extremely metal-poor ($\FeH$$<-3.0$) and well below the metallicity floor of globular clusters, which  supports the growing evidence for the existence of lower-metallicity globular clusters in the early Universe.A third star (P180956,[Fe/H]$\sim-2$) has low[Na,Ca/Mg]and very low[Ba/Fe]for its metallicity, which are consistent with formation in a system polluted by only one or a few low-mass supernovae. Interestingly, its orbit is confined to the Galactic plane, like other very metal-poor stars found in the literature, which have been associated with the earliest building blocks of the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2208.13791/././plots/kine.png" alt="Fig14" width="100%"/>

**Figure 14. -** Orbital parameters of the targets observed with GRACES. The GRACES sample is divided into 4 dynamical groups according to their $r_{\rm apo}$$Z_{\rm max}$. The bulge group is marked with blue stars, the confined group is denoted with blue rhombuses, except for the one with largest $r_{\rm apo}$(magenta rhombus), the inner halo grop with red circles, and the outer halo star with black square. Stars with metallicities below [Fe/H]=$-1.7$  from \citet{Howes14, Howes15, Howes16,Lucey22} are shown as light blue circles. The grey shaded area denotes the forbidden region in which  $Z_{\rm max}$>$r_{\rm apo}$. Vertical and horizontal lines separate the groups as defined in Section \ref{orbsec}.   (*kinefig*)

</div>
<div id="div_fig2">

<img src="tmp_2208.13791/././plots/bah.png" alt="Fig7" width="100%"/>

**Figure 7. -** [Ba/H] vs. $\FeH$. Ba abundances are not corrected by NLTE effect (see Section \ref{nltesec}). The stars in the PIGS/GRACES sample are marked with the same symbols and colours as Figure \ref{kinefig}. The bulge literature sample (blue circles) is composed by stars from \citet{Howes14,Howes15,Howes16,Koch16,Reggiani20,Lucey22}. Halo literature compilation (grey squares) are from \citet{Aoki13,Yong13,Kielty21}. The two Ba-rich stars are C-enhanced stars, therefore they high Ba is produced by the accretion of material from an AGB companion (see Section \ref{cempsec} and \ref{cempgc}. The low-Ba star (magenta diamond marker) is likely accreted from a dwarf galaxy (see Section \ref{planarsec} and Figure \ref{badwarf}). The majority of the sample possesses a [Ba/H] ratio very similar to the MW halo stars at the same $\FeH$(see Section \ref{nonpeculiar}). (*bafig*)

</div>
<div id="div_fig3">

<img src="tmp_2208.13791/././plots/teff_DR3.png" alt="Fig12.1" width="33%"/><img src="tmp_2208.13791/././plots/logg_DR3.png" alt="Fig12.2" width="33%"/><img src="tmp_2208.13791/././plots/met_DR3.png" alt="Fig12.3" width="33%"/>

**Figure 12. -** Comparison of the stellar parameters from  {\it Gaia} DR3. Top panel: Effective temperature. Central panel: Surface gravity. Bottom panel: Metallicity. Blue circles are the stars from this work, black shaded circles and squares are stars from the COMBS \citep{Lucey22} and EMBLA \citep{Howes14,Howes15,Howes16} surveys, respectively. Black lines are the 1:1 comparisons. The agreement between the stellar parameters from {\it Gaia} DR3 and the spectroscopic ones are very poor as expected from the warnings by \citet{Andrae22}. (*gaiadr3_params*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

21  publications in the last 7 days.
	 _build/html/2208.13791.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers